In [4]:
# System level libraries
import os
import subprocess

# Simulation libraries
import openstudio
from pyenergyplus.api import EnergyPlusAPI
import esoreader 

# Data analysis libraries
import pandas as pd
from datetime import timedelta, datetime

# Plotting libraries
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"


In [5]:
output_dir = r"Output_Results"
api = EnergyPlusAPI()

In [6]:
def read_timestep_from_idf(idf_path):
    with open(idf_path, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if line.strip().startswith("Timestep"):
            timestep_line = lines[i + 1].strip()
            timestep_value = timestep_line.split(';')[0].strip()
            return int(timestep_value)

    return None  

def generate_timestamps(timestep, num_records):
    interval_minutes = 60 // timestep
    
    timestamps = []
    current_time = datetime(year=2006, month=1, day=1, hour=0, minute=0, second=0)  
    
    for i in range(num_records + timestep):  
        timestamps.append(current_time)
        current_time += timedelta(minutes=interval_minutes)  
        
    return timestamps[:num_records]  

def run_energyplus_simulation(idf_path, epw_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    state = api.state_manager.new_state()
    api.runtime.set_console_output_status(state, True)

    api.runtime.run_energyplus(state, ['-r', '-d', output_dir, '-w', epw_path, idf_path])
    print("Simulation has successfully finished")
    return output_dir
    

In [7]:
idf_path = r"Sample2.idf"
epw_path = r"ITA_LG_Genoa.161210_TMYx.epw"
output_dir = r"Output_Results"

timestep = read_timestep_from_idf(idf_path)
print(f"Timestep read from IDF file: {timestep}") 

run_energyplus_simulation(idf_path, epw_path, output_dir)

Timestep read from IDF file: 10
Simulation has successfully finished


'Output_Results'

In [16]:
def convert_idf_to_osm(idf_path, osm_output_path):
    # Create a ReverseTranslator object
    reverse_translator = openstudio.energyplus.ReverseTranslator()

    # Load the IDF file into a Workspace object
    idf_file = openstudio.toPath(idf_path)  # Create a Path object
    optional_workspace = openstudio.Workspace.load(idf_file)  # Load Workspace

    if not optional_workspace.is_initialized():
        raise RuntimeError(f"Failed to load the IDF file from {idf_path}")

    workspace = optional_workspace.get()  # Extract the Workspace object

    # Convert the Workspace (IDF) to an OpenStudio Model (OSM)
    model = reverse_translator.translateWorkspace(workspace)  # This returns a Model object

    # Save the converted model to the specified OSM file path
    osm_file_path = openstudio.toPath(osm_output_path)
    success = model.save(osm_file_path, True)  # Directly save the Model object

    if not success:
        raise RuntimeError(f"Failed to save the OSM file to {osm_output_path}")

    print(f"Conversion complete. OSM file saved to {osm_output_path}")

# Example usage
idf_path = "Sample2.idf"
osm_output_path = "Output_Results/converted_model.osm"
convert_idf_to_osm(idf_path, osm_output_path)


Conversion complete. OSM file saved to Output_Results/converted_model.osm


In [17]:
osm_path = openstudio.path(r"Output_Results\converted_model.osm")  
translator = openstudio.osversion.VersionTranslator()
model = translator.loadModel(osm_path)

if model.is_initialized():
    model = model.get()
    print("Model loaded successfully")
else:
    print("Failed to load model")
    model = None

if model:
    vertices_list = []
    i_list = []
    j_list = []
    k_list = []
    colors = []
    face_counter = 0

    spaces = model.getSpaces()  

    for space in spaces:
        for surface in space.surfaces():
            vertices = []
            for vertex in surface.vertices():
                x, y, z = vertex.x(), vertex.y(), vertex.z()
                vertices.append([x, y, z])

            if len(vertices) >= 3:
                start_index = len(vertices_list)
                vertices_list.extend(vertices)

                for n in range(1, len(vertices) - 1):
                    i_list.append(start_index)
                    j_list.append(start_index + n)
                    k_list.append(start_index + n + 1)

                surface_type = surface.surfaceType()
                if surface_type == "Wall":
                    colors.extend([[1.0, 0.8, 0.6]] * (len(vertices) - 2))  
                elif surface_type == "RoofCeiling":
                    colors.extend([[0.7, 0.4, 0.2]] * (len(vertices) - 2))  
                elif surface_type == "Floor" and surface.outsideBoundaryCondition() == "Outdoors":
                    colors.extend([[0.5, 0.5, 0.5]] * (len(vertices) - 2))  
                elif surface_type == "Floor":
                    colors.extend([[0.6, 0.6, 0.6]] * (len(vertices) - 2)) 
                else:
                    colors.extend([[0.7, 0.7, 0.7]] * (len(vertices) - 2))  

            for subsurface in surface.subSurfaces():
                sub_vertices = []
                for vertex in subsurface.vertices():
                    x, y, z = vertex.x(), vertex.y(), vertex.z()
                    sub_vertices.append([x, y, z])

                if len(sub_vertices) >= 3:
                    start_index = len(vertices_list)
                    vertices_list.extend(sub_vertices)

                    for n in range(1, len(sub_vertices) - 1):
                        i_list.append(start_index)
                        j_list.append(start_index + n)
                        k_list.append(start_index + n + 1)

                    subsurface_type = subsurface.subSurfaceType()
                    if subsurface_type == "FixedWindow":
                        colors.extend([[0.3, 0.5, 0.9]] * (len(sub_vertices) - 2))  
                    elif subsurface_type == "Door":
                        colors.extend([[0.7, 0.4, 0.2]] * (len(sub_vertices) - 2))  
                    else:
                        colors.extend([[0.5, 0.5, 0.5]] * (len(sub_vertices) - 2))  

    x, y, z = zip(*vertices_list)
    fig = go.Figure(data=[
        go.Mesh3d(
            x=x, y=y, z=z,
            i=i_list, j=j_list, k=k_list,
            facecolor=colors,
            opacity=0.8,
            flatshading=True
        )
    ])

    fig.update_layout(
        scene=dict(
            xaxis=dict(showbackground=True),
            yaxis=dict(showbackground=True),
            zaxis=dict(showbackground=True),
            aspectmode='data'
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    fig.show()

Model loaded successfully


In [18]:
eso = esoreader.read_from_path(r"Output_Results\eplusout.eso")

In [19]:
eso.find_variable('Site Outdoor Air Drybulb Temperature')

Site_Outdoor_Air_Drybulb_Temperature = eso.to_frame('Site Outdoor Air Drybulb Temperature')

Site_Outdoor_Air_Drybulb_Temperature = Site_Outdoor_Air_Drybulb_Temperature.rename(
    columns={
        'Environment': 'Site_Outdoor_Air_Drybulb_Temperature'
    }
)

num_records = len(Site_Outdoor_Air_Drybulb_Temperature)
timestamps = generate_timestamps(timestep, num_records)

Site_Outdoor_Air_Drybulb_Temperature['Timestamp'] = timestamps
Site_Outdoor_Air_Drybulb_Temperature = Site_Outdoor_Air_Drybulb_Temperature[['Timestamp'] + [col for col in Site_Outdoor_Air_Drybulb_Temperature if col != 'Timestamp']]
Site_Outdoor_Air_Drybulb_Temperature.set_index('Timestamp', inplace=True)

# Daily averages
daily_avg_Site_Outdoor_Air_Drybulb_Temperature = Site_Outdoor_Air_Drybulb_Temperature.resample('D').mean()

# Monthly average
monthly_avg_Site_Outdoor_Air_Drybulb_Temperature = Site_Outdoor_Air_Drybulb_Temperature.resample('M').mean()

In [20]:
daily_avg_Site_Outdoor_Air_Drybulb_Temperature

,Site_Outdoor_Air_Drybulb_Temperature
Timestamp,
2006-01-01,2.216667
2006-01-02,6.615000
2006-01-03,7.038125
2006-01-04,7.171875
2006-01-05,7.804375
...,...
2006-12-27,5.875208
2006-12-28,4.020208
2006-12-29,3.493125


In [21]:
monthly_avg_Site_Outdoor_Air_Drybulb_Temperature

,Site_Outdoor_Air_Drybulb_Temperature
Timestamp,
2006-01-31,5.614543
2006-02-28,7.815885
2006-03-31,9.638972
2006-04-30,14.037035
2006-05-31,17.286022
2006-06-30,20.416875
2006-07-31,22.380134
2006-08-31,23.008723
2006-09-30,19.925639


In [22]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=daily_avg_Site_Outdoor_Air_Drybulb_Temperature.index, y=daily_avg_Site_Outdoor_Air_Drybulb_Temperature['Site_Outdoor_Air_Drybulb_Temperature'],
                         mode='lines', name='Site condition'))

fig.update_layout(
    title={
        'text': 'Average Daily Site Outdoor Air Drybulb Temperature',
        'y': 0.90,  
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    xaxis_title='Date',
    yaxis_title='Drybulb Temperature [C]',
    font=dict(
        family="Times New Roman",
        size=18,
    ),
    xaxis=dict(
        range=['2006-01-01', '2006-12-31'],  
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    ),
    yaxis=dict(fixedrange=False),
    hovermode="x unified",
    paper_bgcolor='white',  
    plot_bgcolor='white',   
    margin=dict(l=60, r=60, t=100, b=60),  
    showlegend=True,
    width=1100,  
    height=600   
)

fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="black", width=2)
        )
    ]
)

fig.show()

In [23]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly_avg_Site_Outdoor_Air_Drybulb_Temperature.index, 
    y=monthly_avg_Site_Outdoor_Air_Drybulb_Temperature['Site_Outdoor_Air_Drybulb_Temperature'],
    mode='lines+markers', 
    name='Site condition',
    line=dict(dash='solid'),  
    marker=dict(symbol='circle', size=8)  
))

month_labels = monthly_avg_Site_Outdoor_Air_Drybulb_Temperature.index.strftime('%b %Y')

fig.update_layout(
    title={
        'text': 'Average Monthly Site Outdoor Air Drybulb Temperature',
        'y': 0.90,  
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    xaxis_title='Date',
    yaxis_title='Drybulb Temperature [C]',
    font=dict(
        family="Times New Roman",
        size=18,
    ),
    xaxis=dict(
        type='category',  
        tickvals=monthly_avg_Site_Outdoor_Air_Drybulb_Temperature.index,
        ticktext=month_labels,
        tickangle=45  
    ),
    yaxis=dict(fixedrange=False),
    hovermode="x unified",
    paper_bgcolor='white',  
    plot_bgcolor='white',   
    margin=dict(l=60, r=60, t=100, b=60),  
    showlegend=True,
    width=1100,  
    height=600   
)

fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="black", width=2)
        )
    ]
)

fig.show()